The goal of this notebook is to find promising areas for mobility hubs. 

To make such decision they would like to consider different features, such as presence of public transport stops, average income of citizens of different districts, share of young population etc. In this notebook I'm just trying to create visualisation that would include the social data about different Hamburg districs as well as data about presence of the public transport in different regions.

Geodata was populated from this website: http://overpass-turbo.eu/

<b>Districts (export_district.geojson):</b>
<pre>    
[out:json][timeout:25];
(
  relation["boundary"="administrative"]["admin_level"="10"]({{bbox}});
);
out body;
>;
out skel qt;
</pre>

<b>Bus stops (export_bus.geojson):</b> 
<pre>
[out:json][timeout:25];
(
  node["highway"="bus_stop"]({{bbox}});
);
out body;
>;
out skel qt;
</pre>

<b>Train Stations (export_bahn.geojson):</b>
<pre> 
[out:json][timeout:25];
(
  node["public_transport"="station"]({{bbox}});
);
out body;
>;
out skel qt;
</pre>

Data about Hamburg (stadt_profile.csv) was extracted from here: 
    http://www.statistik-nord.de/fileadmin/maps/Stadtteil_Profile_2018/atlas.html

Data about passengers load: https://data.deutschebahn.com/dataset/passagierzahlung-s-bahn-hamburg/resource/480dc561-1417-4176-bbd6-8c60f01f6f47

<b>Hamburg Stat Data:</b> 2018 year <br>
<b>Passengers traffic data:</b> 2016-2017 years

In [2]:
%pylab inline

from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium


import datetime
from datetime import timedelta
from h3 import h3

import json
from geojson import Point
from geojson.feature import *
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

Populating the interactive namespace from numpy and matplotlib


Let's create a GeoDataFrame with districts and statistical data

In [3]:
df_districts = gpd.read_file('export_district.geojson')
#remove nulls
df_districts = df_districts[df_districts["name"] == df_districts["name"]]

#Statystical data
df_city_profile = pd.read_csv('stadt_profile.csv', sep = ',')

# Merge GeoDF and statystical, remove empty rows
# remove spaces and lowercase column names and make again a GeoDataFrame
df_districts_all = pd.merge(df_city_profile, df_districts, how='left', on=['name'])
df_districts_all = df_districts_all[df_districts_all["geometry"] == df_districts_all["geometry"]]
df_districts_all.columns = df_districts_all.columns.str.strip().str.lower()
df_districts_all = df_districts_all.set_geometry('geometry')
df_districts_all.crs = {'init' :'epsg:4326'}

/Users/obielov/anaconda3/envs/visual/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Now let's create a DataFrame that will contain info about "density" of public transport

In [4]:
# Bus Stops
df_bus_stops = gpd.read_file('export_bus.geojson')

# S-Bahn stations
df_bahn_stops = gpd.read_file('export_bahn.geojson')

# Passenger flow
df_pass_flow = pd.read_csv("passagierzahlen.csv" ,sep=';',encoding = "ISO-8859-1", 
                        parse_dates=['dtmIstAnkunftDatum', 'dtmIstAbfahrtDatum'])

# Function that gets latitude and longitude 
def getXY(pt):
    return (pt.x, pt.y)

# Getting lat lon from centroids for bus and bahn stops
df_bus_stops["centroid"] = df_bus_stops.centroid
df_bus_stops["lon"], df_bus_stops["lat"] = [list(t) for t in zip(*map(getXY, df_bus_stops["centroid"]))]
df_bahn_stops["centroid"] = df_bahn_stops.centroid
df_bahn_stops["lon"], df_bahn_stops["lat"] = [list(t) for t in zip(*map(getXY, df_bahn_stops["centroid"]))]

# Leave only bahn stops where we know either psv_type or operator. Also filtering out psv_type = A
df_bahn_stops = df_bahn_stops[(df_bahn_stops['hvv:psv_type'] == df_bahn_stops['hvv:psv_type'])
                            | (df_bahn_stops['operator'] == df_bahn_stops['operator'])]
df_bahn_stops = df_bahn_stops[(df_bahn_stops['hvv:psv_type'] != "A")]

# Removing not relevant columns and adding columns that contain 1
# if appropriate transport type operates on that stop
df_bahn_stops = df_bahn_stops[['name', 'operator','hvv:psv_type','lat','lon']]
df_bahn_stops.fillna(value="NaN", inplace=True)
df_bahn_stops['U-Bahn'] = [int('Hochbahn' in x) for x in df_bahn_stops['operator']]
df_bahn_stops['U-Bahn'] = [int('U' in x) for x in df_bahn_stops['hvv:psv_type']]
df_bahn_stops['S-Bahn'] = [int('S-Bahn' in x) for x in df_bahn_stops['operator']]
df_bahn_stops['S-Bahn'] = [int('S' in x) for x in df_bahn_stops['hvv:psv_type']]
df_bahn_stops['DB'] = [int('DB Station' in x) for x in df_bahn_stops['operator']]
df_bahn_stops['R-Bahn'] = [int('R' in x) for x in df_bahn_stops['hvv:psv_type']]
df_bahn_stops['Bus'] = 0
# Remove not needed anymore columns
df_bahn_stops = df_bahn_stops.drop(["operator","hvv:psv_type"], axis = 1)

# Doing the same for Buses
df_bus_stops = df_bus_stops[['name','lat','lon']]
df_bus_stops.fillna(value="NaN", inplace=True)
df_bus_stops['U-Bahn'] = 0
df_bus_stops['S-Bahn'] = 0
df_bus_stops['DB'] = 0
df_bus_stops['R-Bahn'] = 0
df_bus_stops['Bus'] = 1

# Appending buses and bahns in one dataframe
df_transport_all = df_bahn_stops.append(df_bus_stops)
df_transport_all = df_transport_all.reset_index()
df_transport_all = df_transport_all.drop(['index'], axis=1)
df_transport_all.fillna(value="Unknown", inplace=True)

# For DataFrame with passenger flow remove commas 
df_pass_flow["Einsteiger"] = df_pass_flow["Einsteiger"].apply(lambda x: int(x.replace(',', ''))/100)
df_pass_flow["Aussteiger"] = df_pass_flow["Aussteiger"].apply(lambda x: int(x.replace(',', ''))/100)

# Calculating the average daily traffic per station and saving it to df_pass_flow_grouped DataFrame
df_pass_flow['date'] = df_pass_flow.dtmIstAnkunftDatum.dt.floor('d')
df_pass_flow_grouped = df_pass_flow.groupby("Station").sum()
df_pass_flow_grouped["Einsteiger"] = df_pass_flow_grouped["Einsteiger"]/len(df_pass_flow.date.unique())
df_pass_flow_grouped["Aussteiger"] = df_pass_flow_grouped["Aussteiger"]/len(df_pass_flow.date.unique())

# Creating bins based on the traffic. For binning I use Quantile-based discretization (qcut)
# but maybe discrete intervals (cut) would be more correct. Topic for discussion with UX team.
# Also I use 10 bins but this again topic for discussion with UX
df_pass_flow_grouped["bin"] = pd.qcut(df_pass_flow_grouped["Aussteiger"],10, retbins=False,
      labels=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
df_pass_flow_grouped["bin"] = df_pass_flow_grouped["bin"].apply(lambda x: int(x))

# Assigning ranks to S-Bahn stations in df_transport_all DataFrame based on bins created above.
df_transport_all["rank"] = 0
for i, row in df_pass_flow_grouped.iterrows():
    id_ = df_transport_all[df_transport_all["name"].str.contains(str(i)) &
     (df_transport_all["S-Bahn"] == 1)].index
    df_transport_all.at[id_, 'rank'] = row["bin"]


# Size of h3 hexagon. Details here: https://github.com/uber/h3/blob/master/docs/core-library/restable.md
RESOLUTION = 9

# Creating a column with h3 hex_id of each stop
df_transport_all["hex_id"] = df_transport_all.apply(lambda row: h3.geo_to_h3(row["lat"], 
                                                             row["lon"], RESOLUTION), axis = 1)

# Not needed anymore
df_transport_all = df_transport_all.drop(["lat","lon"], axis = 1)

# Removing duplicated stations
df_transport_all.drop_duplicates(subset=['name', 'U-Bahn','S-Bahn','DB',
                                         'R-Bahn','Bus','rank','hex_id'], inplace=True)

# Group all stops based on hexagon they belong to
df_transport_grouped = df_transport_all.groupby("hex_id").sum()

# Moving hex_id out of index
df_transport_grouped = df_transport_grouped.reset_index()


Now let's first join both dataframes: df_transport_grouped and df_districts_all

Resulted dataframe will contain all features - Public Transport related and social information

In [5]:
# finding centroids for each hexagon
df_transport_grouped["centroid"] = df_transport_grouped.hex_id.apply(lambda x: Point(h3.h3_to_geo(h3_address=x)[1],
                                                                                      h3.h3_to_geo(h3_address=x)[0]))
#converting my DataFrame into GeoDataFrmae using that centroids
df_transport_grouped = df_transport_grouped.set_geometry('centroid')
df_transport_grouped.crs = {'init' :'epsg:4326'}

#Merging two dataframes using geodata as a key
df_all_data = gpd.sjoin(left_df = df_transport_grouped, right_df = df_districts_all, how='left', op='intersects')
df_all_data = df_all_data[['hex_id', 'U-Bahn', 'S-Bahn', 'DB', 'R-Bahn', 'Bus', 'rank', 'population',
       'under_18s', 'share_under_18s', '65yo_and_older',
       'share_65yo_and_older', 'foreigners', 'share_foreigners',
       'pop_w_migration_bg', 'share_pop_w_migration_bg', 'under_18_mig_bg',
       'share_under_18_mig_bg', 'hh', 'ppl_per_hh', 'single_per_hh',
       'share_single_per_hh', 'hh_w_chld', 'share_hh_w_chld', 'sngl_parent',
       'share_sngl_parent', 'area_km', 'pop_dnsty', 'births', 'deaths',
       'immigration', 'moving_away', 'migr_balance', 'social_insur',
       'employment_rate', 'unemployed', 'share_unemployed', 'yng_unemployed',
       'share_yng_unemployed', 'old_unemployed', 'share_old_unemployed',
       'sgb_2', 'share_sgb_2', 'under_15_min_security',
       'share_under_15_min_security', 'community_of_needs_sgb_2', 'taxpayers',
       'total_income_per_taxpayer', 'residental_bldngs', 'apartments',
       'ready_to_move_apt', 'apt_in_houses', 'share_apt_in_houses', 'apt_size',
       'living_space_per_inhab', 'social_housing', 'share_social_housing',
       'land_prices', 'kita_preschool', 'primary_schools',
       'share_high_school_stud', 'doctors', 'practitioners', 'dentists',
       'pharmacies', 'private_cars', 'car_density']]

# Removing areas that not belong to Hamburg
df_all_data = df_all_data[df_all_data['car_density'] == df_all_data['car_density']]

df_all_data = df_all_data.reset_index().drop(['index'], axis = 1)

# Creating a geometry column, based on the hex_id
df_all_data["geometry"] =  df_all_data.hex_id.apply(lambda x: 
                                                           {"type" : "Polygon",
                                                            "coordinates": 
                                                            [h3.h3_to_geo_boundary(h3_address=x,geo_json=True)]
                                                            })

Below function plots Polygon areas

In [6]:
def choropleth_map_area(df_aggreg, column_name, border_color = 'none', fill_opacity = 0.35, initial_map = None):
    
    min_value = df_aggreg[column_name].min()
    max_value = df_aggreg[column_name].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    if initial_map is None:
        initial_map = Map(location= [53.552993, 9.989933], zoom_start=11, tiles="cartodbpositron")
    
    custom_cm = cm.LinearColormap(['gray','green','yellow','orange','red'], vmin=min_value, vmax=max_value)
           
    GeoJson(
        df_aggreg,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties'][column_name]),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
    ).add_to(initial_map)
    
    custom_cm.add_to(initial_map)
        
    return initial_map

In [20]:
map_districts = choropleth_map_area(df_aggreg = df_districts_all, column_name = 'total_income_per_taxpayer')
map_districts

Next steps: We could find potentially promising areas using public transport information as well as information in that hexagon, e.g. income, share of teenagers in that hexagon etc.

As a step one let's create a count column that will contain cumulative metric based on the public transport information

In [8]:
# Calculating the cumulative metric for each hexagon which will indicate a
# public transport "density" in that hexagon. Weights are hyperparameters of that cumulative metric
SBAHN_WEIGHT = 4
UBAHN_WEIGHT = 4
RBAHN_WEIGHT = 4
BUS_WEIGHT = 1
df_all_data["counts"] = df_all_data["U-Bahn"] * UBAHN_WEIGHT \
                               + df_all_data["S-Bahn"] * SBAHN_WEIGHT \
                               + df_all_data["R-Bahn"] * RBAHN_WEIGHT \
                               + df_all_data["Bus"] * BUS_WEIGHT \
                               + df_all_data["rank"]


Below functions were taken from somewhere in internet, and what they do is just visualise h3 hexagons.

Actually choropleth_map is just extended version of choropleth_map_area function.

In [9]:
def hexagons_dataframe_to_geojson(df_hex, column_name, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {column_name : row[column_name]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result


def choropleth_map(df_aggreg, column_name, border_color = 'none', fill_opacity = 0.5, initial_map = None, with_legend = True,
                   kind = "outlier"):
    #colormap
    min_value = df_aggreg[column_name].min()
    max_value = df_aggreg[column_name].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [53.552993, 9.989933], zoom_start=11, tiles="cartodbpositron")

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['gray','blue','green','yellow','orange','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['white','gray','black'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   
    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg, column_name = column_name)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties'][column_name]),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)

    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
        
    return initial_map

FYI, below instead of "counts" you can put any column from df_all_data

In [10]:
stops = choropleth_map(df_aggreg = df_all_data, column_name = "counts")
stops

In [11]:
map_all = choropleth_map(df_aggreg = df_all_data, column_name = "counts", initial_map = map_districts)
map_all

Next step could be to add data about the bikes, e.g:<br>
Data: https://data.deutschebahn.com/dataset/data-call-a-bike <br>
Analysis: http://www.alexknowsdata.com/post/stadtrad/

In [12]:
df_bikes_stations = pd.read_csv('bikes_stations.csv', sep=';')

df_bikes_stations = df_bikes_stations[df_bikes_stations["CITY"] == "Hamburg"]
df_bikes_stations = df_bikes_stations[df_bikes_stations["LATITUDE"] == df_bikes_stations["LATITUDE"]]
df_bikes_stations["LATITUDE"] = df_bikes_stations["LATITUDE"].apply(lambda x: float(x.replace(',', '.')))
df_bikes_stations["LONGITUDE"] = df_bikes_stations["LONGITUDE"].apply(lambda x: float(x.replace(',', '.')))
df_bikes_stations["hex_id"] = df_bikes_stations.apply(lambda row: h3.geo_to_h3(row["LONGITUDE"], 
                                                             row["LATITUDE"], RESOLUTION), axis = 1)
df_bikes_stations = df_bikes_stations[['NAME','RENTAL_ZONE_HAL_ID','hex_id']]

df_bikes_stations = df_bikes_stations[['RENTAL_ZONE_HAL_ID','hex_id']].set_index('RENTAL_ZONE_HAL_ID')

stations_dict = df_bikes_stations.to_dict()['hex_id']

In [13]:
DATE_FROM = datetime.datetime.strptime("2017-04-01", "%Y-%m-%d")
DATE_TO = datetime.datetime.strptime("2017-05-01", "%Y-%m-%d")

df_bikes_bookings = pd.read_csv('bikes_bookings.csv', sep=';',
                                parse_dates=['DATE_BOOKING'])

df_bikes_bookings['DATE_BOOKING'] = df_bikes_bookings['DATE_BOOKING'].dt.floor('d')
df_bikes_bookings = df_bikes_bookings[df_bikes_bookings['CITY_RENTAL_ZONE'] == 'Hamburg']
df_bikes_bookings = df_bikes_bookings[['START_RENTAL_ZONE_HAL_ID','END_RENTAL_ZONE_HAL_ID',
                                      'DATE_BOOKING']]

df_bikes_bookings = df_bikes_bookings[df_bikes_bookings["START_RENTAL_ZONE_HAL_ID"] == df_bikes_bookings["START_RENTAL_ZONE_HAL_ID"]]
df_bikes_bookings = df_bikes_bookings[df_bikes_bookings["END_RENTAL_ZONE_HAL_ID"] == df_bikes_bookings["END_RENTAL_ZONE_HAL_ID"]]

df_bikes_bookings["START_RENTAL_ZONE_HAL_ID"] = df_bikes_bookings["START_RENTAL_ZONE_HAL_ID"].apply(lambda x: int(x))
df_bikes_bookings["END_RENTAL_ZONE_HAL_ID"] = df_bikes_bookings["END_RENTAL_ZONE_HAL_ID"].apply(lambda x: int(x))

df_bikes_bookings = df_bikes_bookings[(df_bikes_bookings["DATE_BOOKING"] >= DATE_FROM)
                   & (df_bikes_bookings["DATE_BOOKING"] < DATE_TO)]

df_bikes_bookings = df_bikes_bookings.reset_index()
df_bikes_bookings = df_bikes_bookings.drop(["index"], axis = 1)

df_bikes_bookings = df_bikes_bookings[df_bikes_bookings['START_RENTAL_ZONE_HAL_ID'].isin(df_bikes_stations.index) == True]
df_bikes_bookings = df_bikes_bookings[df_bikes_bookings['END_RENTAL_ZONE_HAL_ID'].isin(df_bikes_stations.index) == True]


df_bikes_bookings['hex_start'] = df_bikes_bookings['START_RENTAL_ZONE_HAL_ID'].apply(lambda x: stations_dict[x])
df_bikes_bookings['hex_end'] = df_bikes_bookings['END_RENTAL_ZONE_HAL_ID'].apply(lambda x: stations_dict[x])


df_bikes_bookings = df_bikes_bookings[['DATE_BOOKING','hex_start','hex_end']]

df_bikes_bookings_start = df_bikes_bookings.groupby('hex_start').count()
df_bikes_bookings_start.index = df_bikes_bookings_start.index.rename('hex_id')
df_bikes_bookings_start = df_bikes_bookings_start.reset_index()

df_bikes_bookings_end = df_bikes_bookings.groupby('hex_end').count()
df_bikes_bookings_end.index = df_bikes_bookings_end.index.rename('hex_id')
df_bikes_bookings_end = df_bikes_bookings_end.reset_index()

df_bikes_bookings = pd.merge(df_bikes_bookings_start, df_bikes_bookings_end, how='inner', on=['hex_id'])
df_bikes_bookings = df_bikes_bookings[['hex_id','hex_end','hex_start']]

df_bikes_bookings = df_bikes_bookings.rename(columns={"hex_start": "bike_start", "hex_end": "bike_end"})


df_bikes_bookings["geometry"] =  df_bikes_bookings.hex_id.apply(lambda x: 
                                                           {"type" : "Polygon",
                                                            "coordinates": 
                                                            [h3.h3_to_geo_boundary(h3_address=x,geo_json=True)]
                                                            })

In [14]:
bike = choropleth_map(df_aggreg = df_bikes_bookings, column_name = "bike_start")
bike

In [15]:
bike = choropleth_map(df_aggreg = df_bikes_bookings[df_bikes_bookings['hex_id'].isin(df_all_data.hex_id) == False], column_name = "bike_start")
bike



In [16]:
df_bikes_bookings.to_csv("df_bikes_bookings.csv")

In [17]:
df_all_data.to_csv("df_all_data.csv")

In [18]:
df_districts_all.to_csv("df_districts_all.csv")

In [19]:
df_all_data = pd.merge(df_all_data, df_bikes_bookings.drop(['geometry'], axis = 1), how='inner', on=['hex_id'])

In [19]:
df_all_data.to_csv("df_all_data_hex_9.csv")

In [21]:
df_all_data.head()

,hex_id,U-Bahn,S-Bahn,DB,R-Bahn,Bus,rank,population,under_18s,share_under_18s,...,doctors,practitioners,dentists,pharmacies,private_cars,car_density,geometry,counts,bike_end,bike_start
0,891f1530323ffff,0,0,0,0,2,0,25860.0,4081.0,15.781129,...,178.0,34.0,62.0,12.0,4850.0,187.548337,"{'type': 'Polygon', 'coordinates': [[[9.974082...",2,476,472
1,891f1530327ffff,0,1,0,0,2,10,25860.0,4081.0,15.781129,...,178.0,34.0,62.0,12.0,4850.0,187.548337,"{'type': 'Polygon', 'coordinates': [[[9.979120...",16,400,392
2,891f153032fffff,0,0,0,0,1,0,25860.0,4081.0,15.781129,...,178.0,34.0,62.0,12.0,4850.0,187.548337,"{'type': 'Polygon', 'coordinates': [[[9.976915...",1,329,310
3,891f153033bffff,0,0,0,0,1,0,25860.0,4081.0,15.781129,...,178.0,34.0,62.0,12.0,4850.0,187.548337,"{'type': 'Polygon', 'coordinates': [[[9.969045...",1,736,717
4,891f1530353ffff,0,0,0,0,1,0,22082.0,3810.0,17.253872,...,27.0,9.0,12.0,3.0,6308.0,285.662531,"{'type': 'Polygon', 'coordinates': [[[9.948898...",1,139,119
